In [1]:
import re

import matplotlib.pyplot as plt
% matplotlib inline

import pandas as pd
from keras import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import np
from keras.optimizers import RMSprop
from keras.utils import np_utils
from numpy import array
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [2]:
np.random.seed(1337)  # for reproducibility

def extract_max(input):
    return list(map(int, re.findall('\d+', input)))

def kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

In [3]:
train_df = pd.read_csv("../all/train.csv")
test_df = pd.read_csv("../all/test/test.csv")

In [4]:
train_df.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,...,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


In [5]:
cat_cols = ['Type','Age','Breed1','Breed2', 'Gender', 'Color1', 'Color2', 'Color3', 'MaturitySize', 
          'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized','Health', 'Quantity','State','VideoAmt','PhotoAmt']
num_cols = ['Fee']
text_cols = ['Description']

## Scale data

In [6]:
print('scaling num_cols')
for col in num_cols:
    print('scaling {}'.format(col))
    col_mean = train_df[col].mean()
    train_df[col].fillna(col_mean, inplace=True)
    test_df[col].fillna(col_mean, inplace=True)
    scaler = StandardScaler()
    train_df[col] = scaler.fit_transform(train_df[col].values.reshape(-1, 1))
    test_df[col] = scaler.transform(test_df[col].values.reshape(-1, 1))

tr_df, val_df = train_test_split(train_df, test_size=0.5, random_state=4)

y_train = np_utils.to_categorical(tr_df['AdoptionSpeed'], num_classes=5)
y_valid = np_utils.to_categorical(val_df['AdoptionSpeed'], num_classes=5)

cs = StandardScaler()
trainContinuous = cs.fit_transform(tr_df[cat_cols])
trainContinuous2 = cs.fit_transform(val_df[cat_cols])
trainContinuous3 = cs.fit_transform(test_df[cat_cols])
x_train = np.hstack([trainContinuous])
x_valid = np.hstack([trainContinuous2])
x_test = np.hstack([trainContinuous3])

scaling num_cols
scaling Fee


/Users/viteka/final_project/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/viteka/final_project/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/viteka/final_project/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/viteka/final_project/venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/viteka/final_project/venv/lib/python3.6/site-packages/skl

## Define NN Model

In [7]:
model = Sequential([
        Dense(64, input_dim=18),
        Activation('relu'),
        Dense(32),
        Activation('relu'),
        Dense(32),
        Activation('relu'),
        Dense(16),
        Activation('relu'),
        Dense(5),
        Activation('sigmoid'),
    ])

rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# We add metrics to get more results you want to see
# categorical_crossentropy--mse
model.compile(optimizer=rmsprop,
              loss='mse',
              metrics=['accuracy'])

## Train NN

In [8]:
filepath = "../checkpoints/weights_image_categorical.hdf6"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
earlystopped = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=10, verbose=0, mode='max')
callbacks_list = [checkpoint]

for i in range(10):
    x_train1, x_train2, y_train1, y_train2 = train_test_split(x_train, y_train, test_size=0.2, random_state=i * 15)
    history = model.fit(x_train1, y_train1, validation_data=(x_train2, y_train2), epochs=100, batch_size=1000,
                        shuffle=True, callbacks=callbacks_list)

Train on 5996 samples, validate on 1500 samples
Epoch 1/100
5996/5996 [==============================] - 0s 44us/step - loss: 0.2318 - acc: 0.1788 - val_loss: 0.2097 - val_acc: 0.1907

Epoch 00001: val_acc improved from -inf to 0.19067, saving model to ../checkpoints/weights_image_categorical.hdf6
Epoch 2/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1997 - acc: 0.2050 - val_loss: 0.1876 - val_acc: 0.1987

Epoch 00002: val_acc improved from 0.19067 to 0.19867, saving model to ../checkpoints/weights_image_categorical.hdf6
Epoch 3/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1814 - acc: 0.2100 - val_loss: 0.1750 - val_acc: 0.2093

Epoch 00003: val_acc improved from 0.19867 to 0.20933, saving model to ../checkpoints/weights_image_categorical.hdf6
Epoch 4/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1709 - acc: 0.2260 - val_loss: 0.1671 - val_acc: 0.2200

Epoch 00004: val_acc improved from 0.20933 to 0.22000, savi

5996/5996 [==============================] - 0s 3us/step - loss: 0.1388 - acc: 0.4191 - val_loss: 0.1449 - val_acc: 0.3733

Epoch 00038: val_acc did not improve from 0.38200
Epoch 39/100
5996/5996 [==============================] - 0s 4us/step - loss: 0.1384 - acc: 0.4221 - val_loss: 0.1449 - val_acc: 0.3733

Epoch 00039: val_acc did not improve from 0.38200
Epoch 40/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1385 - acc: 0.4223 - val_loss: 0.1458 - val_acc: 0.3707

Epoch 00040: val_acc did not improve from 0.38200
Epoch 41/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1383 - acc: 0.4194 - val_loss: 0.1456 - val_acc: 0.3713

Epoch 00041: val_acc did not improve from 0.38200
Epoch 42/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1381 - acc: 0.4258 - val_loss: 0.1452 - val_acc: 0.3693

Epoch 00042: val_acc did not improve from 0.38200
Epoch 43/100
5996/5996 [==============================] - 0s 3us/step - loss: 

5996/5996 [==============================] - 0s 3us/step - loss: 0.1332 - acc: 0.4545 - val_loss: 0.1463 - val_acc: 0.3733

Epoch 00082: val_acc did not improve from 0.38200
Epoch 83/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1330 - acc: 0.4565 - val_loss: 0.1465 - val_acc: 0.3673

Epoch 00083: val_acc did not improve from 0.38200
Epoch 84/100
5996/5996 [==============================] - 0s 4us/step - loss: 0.1333 - acc: 0.4601 - val_loss: 0.1462 - val_acc: 0.3627

Epoch 00084: val_acc did not improve from 0.38200
Epoch 85/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1328 - acc: 0.4608 - val_loss: 0.1465 - val_acc: 0.3680

Epoch 00085: val_acc did not improve from 0.38200
Epoch 86/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1331 - acc: 0.4595 - val_loss: 0.1473 - val_acc: 0.3713

Epoch 00086: val_acc did not improve from 0.38200
Epoch 87/100
5996/5996 [==============================] - 0s 3us/step - loss: 

5996/5996 [==============================] - 0s 3us/step - loss: 0.1312 - acc: 0.4655 - val_loss: 0.1378 - val_acc: 0.4380

Epoch 00025: val_acc did not improve from 0.47067
Epoch 26/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1308 - acc: 0.4688 - val_loss: 0.1394 - val_acc: 0.4260

Epoch 00026: val_acc did not improve from 0.47067
Epoch 27/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1310 - acc: 0.4733 - val_loss: 0.1376 - val_acc: 0.4320

Epoch 00027: val_acc did not improve from 0.47067
Epoch 28/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1306 - acc: 0.4721 - val_loss: 0.1383 - val_acc: 0.4307

Epoch 00028: val_acc did not improve from 0.47067
Epoch 29/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1310 - acc: 0.4665 - val_loss: 0.1392 - val_acc: 0.4253

Epoch 00029: val_acc did not improve from 0.47067
Epoch 30/100
5996/5996 [==============================] - 0s 3us/step - loss: 

5996/5996 [==============================] - 0s 3us/step - loss: 0.1280 - acc: 0.4847 - val_loss: 0.1404 - val_acc: 0.4280

Epoch 00069: val_acc did not improve from 0.47067
Epoch 70/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1260 - acc: 0.5012 - val_loss: 0.1400 - val_acc: 0.4300

Epoch 00070: val_acc did not improve from 0.47067
Epoch 71/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1264 - acc: 0.4922 - val_loss: 0.1435 - val_acc: 0.4000

Epoch 00071: val_acc did not improve from 0.47067
Epoch 72/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1278 - acc: 0.4872 - val_loss: 0.1407 - val_acc: 0.4200

Epoch 00072: val_acc did not improve from 0.47067
Epoch 73/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1257 - acc: 0.5053 - val_loss: 0.1401 - val_acc: 0.4293

Epoch 00073: val_acc did not improve from 0.47067
Epoch 74/100
5996/5996 [==============================] - 0s 3us/step - loss: 

...

5996/5996 [==============================] - 0s 3us/step - loss: 0.1038 - acc: 0.6159 - val_loss: 0.1159 - val_acc: 0.5647

Epoch 00023: val_acc did not improve from 0.60600
Epoch 24/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1017 - acc: 0.6263 - val_loss: 0.1173 - val_acc: 0.5680

Epoch 00024: val_acc did not improve from 0.60600
Epoch 25/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1041 - acc: 0.6119 - val_loss: 0.1172 - val_acc: 0.5653

Epoch 00025: val_acc did not improve from 0.60600
Epoch 26/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1031 - acc: 0.6179 - val_loss: 0.1161 - val_acc: 0.5607

Epoch 00026: val_acc did not improve from 0.60600
Epoch 27/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1019 - acc: 0.6269 - val_loss: 0.1168 - val_acc: 0.5620

Epoch 00027: val_acc did not improve from 0.60600
Epoch 28/100
5996/5996 [==============================] - 0s 3us/step - loss: 

5996/5996 [==============================] - 0s 3us/step - loss: 0.0994 - acc: 0.6369 - val_loss: 0.1245 - val_acc: 0.5227

Epoch 00067: val_acc did not improve from 0.60600
Epoch 68/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1021 - acc: 0.6237 - val_loss: 0.1238 - val_acc: 0.5293

Epoch 00068: val_acc did not improve from 0.60600
Epoch 69/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1001 - acc: 0.6314 - val_loss: 0.1215 - val_acc: 0.5393

Epoch 00069: val_acc did not improve from 0.60600
Epoch 70/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1001 - acc: 0.6311 - val_loss: 0.1250 - val_acc: 0.5247

Epoch 00070: val_acc did not improve from 0.60600
Epoch 71/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1006 - acc: 0.6308 - val_loss: 0.1229 - val_acc: 0.5393

Epoch 00071: val_acc did not improve from 0.60600
Epoch 72/100
5996/5996 [==============================] - 0s 3us/step - loss: 

5996/5996 [==============================] - 0s 3us/step - loss: 0.1003 - acc: 0.6373 - val_loss: 0.1100 - val_acc: 0.5907

Epoch 00010: val_acc did not improve from 0.62600
Epoch 11/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1045 - acc: 0.6147 - val_loss: 0.1107 - val_acc: 0.5880

Epoch 00011: val_acc did not improve from 0.62600
Epoch 12/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1015 - acc: 0.6276 - val_loss: 0.1087 - val_acc: 0.5993

Epoch 00012: val_acc did not improve from 0.62600
Epoch 13/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1033 - acc: 0.6221 - val_loss: 0.1096 - val_acc: 0.5967

Epoch 00013: val_acc did not improve from 0.62600
Epoch 14/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.1003 - acc: 0.6384 - val_loss: 0.1099 - val_acc: 0.5947

Epoch 00014: val_acc did not improve from 0.62600
Epoch 15/100
5996/5996 [==============================] - 0s 3us/step - loss: 

5996/5996 [==============================] - 0s 3us/step - loss: 0.0990 - acc: 0.6424 - val_loss: 0.1202 - val_acc: 0.5533

Epoch 00054: val_acc did not improve from 0.62600
Epoch 55/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.0996 - acc: 0.6408 - val_loss: 0.1179 - val_acc: 0.5640

Epoch 00055: val_acc did not improve from 0.62600
Epoch 56/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.0996 - acc: 0.6391 - val_loss: 0.1178 - val_acc: 0.5567

Epoch 00056: val_acc did not improve from 0.62600
Epoch 57/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.0971 - acc: 0.6533 - val_loss: 0.1184 - val_acc: 0.5593

Epoch 00057: val_acc did not improve from 0.62600
Epoch 58/100
5996/5996 [==============================] - 0s 2us/step - loss: 0.0998 - acc: 0.6374 - val_loss: 0.1198 - val_acc: 0.5473

Epoch 00058: val_acc did not improve from 0.62600
Epoch 59/100
5996/5996 [==============================] - 0s 3us/step - loss: 

5996/5996 [==============================] - 0s 3us/step - loss: 0.0967 - acc: 0.6531 - val_loss: 0.1237 - val_acc: 0.5387

Epoch 00098: val_acc did not improve from 0.62600
Epoch 99/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.0980 - acc: 0.6473 - val_loss: 0.1230 - val_acc: 0.5313

Epoch 00099: val_acc did not improve from 0.62600
Epoch 100/100
5996/5996 [==============================] - 0s 3us/step - loss: 0.0967 - acc: 0.6541 - val_loss: 0.1222 - val_acc: 0.5333

Epoch 00100: val_acc did not improve from 0.62600


## Test NN

In [9]:
model.load_weights('../checkpoints/bestForNow.hdf6')

print('\nTesting ------------')
# Evaluate the model with the metrics we defined earlier
loss = model.predict(x_train, batch_size=1000)
ans = [0 for x in range(len(x_train))]

for i in range(len(x_train)):
    index_of_maximum = np.where(loss[i] == loss[i].max())
    ans[i] = extract_max(str(index_of_maximum))

ans = array(ans)

y_train = tr_df['AdoptionSpeed'].values

loss1 = model.predict(x_valid, batch_size=1000)
ans1 = [0 for x in range(len(x_valid))]

for i in range(len(x_valid)):
    index_of_maximum = np.where(loss1[i] == loss1[i].max())
    ans1[i] = extract_max(str(index_of_maximum))

ans1 = array(ans1)

y_valid = val_df['AdoptionSpeed'].values

y_train_pred = ans
y_valid_pred = ans1
avg_train_kappa = 0
avg_valid_kappa = 0
avg_train_kappa += kappa(y_train_pred, y_train)
avg_valid_kappa += kappa(y_valid_pred, y_valid)
print("\navg train kappa:", avg_train_kappa, )
print("\navg valid kappa:", avg_valid_kappa, )


Testing ------------

avg train kappa: 0.49405957902509934

avg valid kappa: 0.4881544322433451


In [10]:
test_pred = model.predict(x_test)
kappa(y_test, test_pred)

NameError: name 'y_test' is not defined

In [ ]:
train_pred = model.predict(x_train)
kappa(y_train, train_pred)

## Print results

In [ ]:
f = plt.figure(figsize=(10, 3))
ax = f.add_subplot(121)
ax2 = f.add_subplot(122)

ax.plot(history.history['loss'])
ax.plot(history.history['val_loss'])
ax.set_title('Model loss')
ax.set_xlabel('epoch')
ax.legend(['train', 'valid'], loc='upper left')

ax2.plot(history.history['acc'])
ax2.plot(history.history['val_acc'])
ax2.legend(['acc', 'val_acc'])
ax2.plot()